# Fine stable diffusion XL

In [ ]:
import ads
import oci
import os
import ocifs
from oci.object_storage import ObjectStorageClient
from datetime import datetime, timedelta

ads.set_auth(auth='resource_principal')
rps = oci.auth.signers.get_resource_principals_signer()

# Object storage client
object_storage_client = ObjectStorageClient(config={}, signer=rps)
OBJECT_STORAGE_NAMESPACE = object_storage_client.get_namespace().data

os.environ['namespace'] = OBJECT_STORAGE_NAMESPACE
LOG_GROUP_ID = os.environ['loggroup_ocid']
LOG_ID = os.environ['log_ocid']
BUCKET_NAME = os.environ['bucket_name']

INPUT_FOLDER = "oci://{bucket}@{namespace}/sdxl/input/".format(bucket=BUCKET_NAME, namespace=OBJECT_STORAGE_NAMESPACE)
OUTPUT_FOLDER = "oci://{bucket}@{namespace}/sdxl/output/".format(bucket=BUCKET_NAME, namespace=OBJECT_STORAGE_NAMESPACE)

CONDA_ENV = "oci://{bucket}@{namespace}/conda_environments/cpu/ONNX 1.10 for CPU on Python 3.9/1.0/onnx110_p39_cpu_v1".format(bucket=BUCKET_NAME, namespace=OBJECT_STORAGE_NAMESPACE)

In [ ]:
!git clone https://github.com/bmaltais/kohya_ss.git /home/datascience/job_artifact/kohya_ss
!odsc conda install -s onnx110_p39_cpu_v1
!conda init bash
!source ~/.bashrc && conda activate /home/datascience/conda/onnx110_p39_cpu_v1 && cd /home/datascience/job_artifact/kohya_ss && bash setup.sh 
!odsc conda init -b $bucket_name -n $namespace -a resource_principal
!rm -rf /home/datascience/conda/onnx110_p39_cpu_v1/lib/python3.9/site-packages/library.pth
!odsc conda publish -s onnx110_p39_cpu_v1 --force
!cp /home/datascience/repos/carlgira/oci-tf-odsc-sdxl/app/stable_main.py /home/datascience/job_artifact/

In [ ]:
from ads.jobs import Job, DataScienceJob, PythonRuntime

job = (
    Job(name="sdxl-train-job")
    .with_infrastructure(
        DataScienceJob()
        .with_log_group_id(LOG_GROUP_ID)
        .with_log_id(LOG_ID)
        .with_shape_name("VM.GPU2.1")
    )
    .with_runtime(
        PythonRuntime()
        .with_custom_conda(CONDA_ENV)
        .with_source("/home/datascience/job_artifact/")
        .with_entrypoint("stable_main.py")
        .with_working_dir("job_artifact")
        .with_environment_variable(full_input_folder=INPUT_FOLDER)
        .with_output("./output", OUTPUT_FOLDER)
    )
)

job.create()

In [ ]:
job_run_env = job.run(
    name="Job Run - Passing dynamic values",
    env_var={'full_input_folder': INPUT_FOLDER}
)
#job_run_watch = job_run_env.watch()

In [ ]:
import os
from ads.model.generic_model import GenericModel

fs = ocifs.OCIFileSystem()

print(1)
if not os.path.exists("/home/datascience/model_artifact"):
    !git clone https://github.com/comfyanonymous/ComfyUI /home/datascience/model_artifact

print(2)
path_to_model_artifacts = "/home/datascience/model_artifact/"
generic_model = GenericModel(artifact_dir=path_to_model_artifacts)
generic_model.summary_status()

# Autogenerate score.py, pickled model, runtime.yaml, input_schema.json and output_schema.json
generic_model.prepare(
        inference_conda_env=CONDA_ENV,
        model_file_name="sks.safetensors",
        force_overwrite=True)


print(3)
if not os.path.exists("/home/datascience/model_artifact/sks.safetensors"):
    fs.invalidate_cache(OUTPUT_FOLDER)
    fs.get(OUTPUT_FOLDER + "sks.safetensors", path_to_model_artifacts , recursive=True, refresh=True)

print(4)
!cp /home/datascience/repos/carlgira/oci-tf-odsc-sdxl/app/score.py /home/datascience/model_artifact



In [ ]:
project_id = os.environ['PROJECT_OCID'] 
compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']

model_id = generic_model.save(display_name="sdxl-fine-tuned", timeout=1800)

In [ ]:
generic_model.deploy(
    display_name="sdxl-model",
    deployment_log_group_id=LOG_GROUP_ID,
    deployment_predict_log_id=LOG_ID,
    deployment_instance_shape="VM.GPU2.1",
)

In [ ]:
generic_model.predict("prompt")